In [38]:
import torch
from time import perf_counter
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torch.utils
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
%matplotlib inline  
print(torch.cuda.is_available())
cifar_mean_stddev = ((0.49139968,  0.48215841,  0.44653091), (0.24703223,  0.24348513,  0.26158784))
train_ratio=0.8
batch_size=64

False


In [39]:
#UTILS

import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

class Private_Image_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        load_data = np.load(data_dir+"\private_test_images_v3.npy")
        self.data=load_data.reshape(2000, 32, 32, 3)
        self.transform = transform

    def __getitem__(self, index):
        img = self.data[index]
        x = Image.fromarray(img)
        x = self.transform(x)

        return x

    def __len__(self):
        return len(self.data)

cifar_mean_stddev = ((0.49139968,  0.48215841,  0.44653091), (0.24703223,  0.24348513,  0.26158784))

def display_batch(dataset_loaded, test=False):
    (mean, stddev)=cifar_mean_stddev
    mean = torch.tensor(mean).reshape(1,3,1,1)
    stddev = torch.tensor(stddev).reshape(1,3,1,1)
    if test:
        for images in dataset_loaded:
            fig, ax = plt.subplots(figsize = (10,10))
            images = images * stddev + mean
            ax.imshow(make_grid(images,nrow=10).permute(1,2,0))
            fig.savefig('test_batch.png', dpi=200) 
            break
    else: 
        for images, labels in dataset_loaded:
            fig, ax = plt.subplots(figsize = (10,10))
            images = images * stddev + mean
            ax.imshow(make_grid(images,10).permute(1,2,0))
            fig.savefig('batch.png', dpi=200) 
            print(labels)
            break

#Reference: https://pytorch.org/tutorials/beginner/nn_tutorial.html#wrapping-dataloader

def get_torch_device():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    return device

def move_to_device(object, device):
    if (isinstance(object, (list, tuple))):
        # tensor = tf.convert_to_tensor(array)
        return [move_to_device(obj, device) for obj in object]
    elif isinstance(object, dict):
        return [(k, move_to_device(v, device)) for k,v in object.items]
    else:
        return object.to(device=device, non_blocking = True)

class WrappedDataLoader:
    def __init__(self, dataset_loaded):
        self.dataset_loaded = dataset_loaded

    def __len__(self):
        return len(self.dataset_loaded)

    def __iter__(self):
        batches = iter(self.dataset_loaded)
        for b in batches:
            yield (move_to_device(b, device=get_torch_device()))

In [36]:
#IMAGEUTILS

import numpy as np
import torchvision.transforms as transforms

"""This script implements the functions for data augmentation
and preprocessing.
"""

#UNUSED METHOD
def parse_record(record, training):
    """Parse a record to an image and perform data preprocessing.

    Args:
        record: An array of shape [3072,]. One row of the x_* matrix.
        training: A boolean. Determine whether it is in training mode.

    Returns:
        image: An array of shape [32, 32, 3].
    """
    ### YOUR CODE HERE
    
    # Reshape from [depth * height * width] to [depth, height, width].
    depth_major = record.reshape((3, 32, 32))

    # Convert from [depth, height, width] to [height, width, depth]
    image = np.transpose(depth_major, [1, 2, 0])

    image = preprocess_image(image, training)

    # Convert from [height, width, depth] to [depth, height, width]
    image = np.transpose(image, [2, 0, 1])

    ### END CODE HERE

    image = preprocess_image(image, training) # If any.

    return image


def preprocess_image():
    """Preprocess a single image of shape [height, width, depth].

    Args:
        image: An array of shape [32, 32, 3].
        training: A boolean. Determine whether it is in training mode.

    Returns:
        preprocess_train, preprocess_test: Transformations for train and test preprocessing.
    """
    ### YOUR CODE HERE
    #Reference for stats values: https://github.com/Armour/pytorch-nn-practice/blob/master/utils/meanstd.py
    cifar_mean_stddev = ((0.49139968,  0.48215841,  0.44653091), (0.24703223,  0.24348513,  0.26158784))
    #Tutorial for using pytorch transforms:https://pytorch.org/vision/stable/auto_examples/plot_scripted_tensor_transforms.html
    preprocess_train = transforms.Compose([
        transforms.RandomCrop(32,padding = 4, padding_mode = 'reflect'),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(*cifar_mean_stddev, inplace=True) 
    ])
    
    preprocess_test = transforms.Compose([
         transforms.ToTensor(),
        transforms.Normalize(*cifar_mean_stddev, inplace=True)
    ])

    ### END CODE HERE

    return (preprocess_train, preprocess_test)

In [37]:
#DATALOADER

from Utils import Private_Image_Dataset, WrappedDataLoader, display_batch
from ImageUtils import preprocess_image
import torchvision
import torch
from torch.utils.data import DataLoader

"""This script implements the functions for reading data.
"""
batch_size=64

def load_data(data_dir=''):
    """Load the CIFAR-10 dataset.

    Args:
        data_dir: A string. The directory where data batches
            are stored/will be downloaded.

    Returns:
        train_dataset_loaded: Training Dataset
        valid_dataset_loaded: Validation Dataset
        cifar_test_dataset_loaded: CIFAR-10 Test Dataset
    """
    
    input_train_dataset = torchvision.datasets.CIFAR10(root = data_dir, download = True, transform = preprocess_image()[0])
    cifar_test_dataset = torchvision.datasets.CIFAR10(root = data_dir, download = True, transform = preprocess_image()[1], train = False)
    
    train_dataset, valid_dataset = train_valid_split(input_train_dataset)
    train_dataset_loaded = DataLoader(train_dataset, batch_size, shuffle = True, pin_memory = True)
    valid_dataset_loaded = DataLoader(valid_dataset, batch_size, shuffle = True, pin_memory = True)
    cifar_test_dataset_loaded = DataLoader(cifar_test_dataset, batch_size, shuffle = True, pin_memory = True)
    
    train_dataset_loaded = WrappedDataLoader(train_dataset_loaded)
    valid_dataset_loaded = WrappedDataLoader(valid_dataset_loaded)
    cifar_test_dataset_loaded = WrappedDataLoader(cifar_test_dataset_loaded)
    display_batch(cifar_test_dataset_loaded)
    ### END CODE HERE

    return train_dataset_loaded, valid_dataset_loaded, cifar_test_dataset_loaded

def load_private_testing_images(data_dir):
    """Load the images in private testing dataset.

    Args:
        data_dir: A string. The directory where the testing images
        are stored.

    Returns:
        private_test_dataset_loaded: Private Test dataset
    """

    ### YOUR CODE HERE
    private_test_dataset=Private_Image_Dataset(data_dir, transform=preprocess_image()[1])
    private_test_dataset_loaded = DataLoader(private_test_dataset, batch_size, shuffle = False, pin_memory = True, num_workers=2)
    private_test_dataset_loaded = WrappedDataLoader(private_test_dataset_loaded)
    display_batch(private_test_dataset_loaded, test=True)
    ### END CODE HERE

    return private_test_dataset_loaded

def train_valid_split(input_train_dataset, train_ratio=0.8):
    """Split the original training data into a new training dataset
    and a validation dataset.

    Args:
        input_train_dataset: Input training dataset
        train_ratio: A float number between 0 and 1.

    Returns:
        train_dataset: Training dataset of length train_ratio*x_train.shape.
        valid_dataset: Validation dataset of length [1-train_ratio]*x_train.shape.
    """
    
    ### YOUR CODE HERE
    split_index=int(train_ratio* len(input_train_dataset))
    train_dataset, valid_dataset = torch.utils.data.random_split(
        dataset = input_train_dataset, 
        lengths = [split_index, len(input_train_dataset)-split_index],
        generator = torch.Generator().manual_seed(42))
    ### END CODE HERE

    return train_dataset, valid_dataset